In [1]:
import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=ZrYbP7BgUVXB0vqvaFyeMlsKZR88BNjY6cT2rJbTl7E&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjW03aJFrBeqZG9wWy8DLcNa5u9ukW7Th_GBMcf6HBgdyYLGbWN_E4

Successfully saved authorization token.


Get surface reflectance from Sentinel and Landsat satellites. These have been atmospherically corrected to obtain the surface reflectance from imaged top of atmosphere radiance using standard algorithms.

User needs to specify polygon of interest and time frame of interest.

Prints image bands and total number of images within a time frame.

Sensor info: Landsat-8 2013-present. Landsat-7 1999-present. Sentinel-2 2015-present.

The earth engine data catalog is found: https://developers.google.com/earth-engine/datasets/catalog


In [2]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
%matplotlib inline

# create polygon for region of interest (WGS84 coordindates)
# currently centered on Port of Napier
#polygon = ee.Geometry.Polygon([[176.854477,-39.405958], [177.034721,-39.408876], [177.050858,-39.505630], [176.831818,-39.511193]]);
polygon = ee.Geometry.Polygon([[-123.7118, 42.6437], [-123.7118, 42.99299], [-123.4263,42.99299],[-123.4263,42.6437]]);

# image collections
# l8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") # Landsat 8 level 2
# s2 = ee.ImageCollection("COPERNICUS/S2_SR") # Sentinel 2 level 2
# l7 = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR") # Landsat-7 surface reflectance (level2)
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") # Landsat 8 level 2
#s2 = ee.ImageCollection("COPERNICUS/S2_SR") # Sentinel 2 level 2
l7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2") # Landsat-7 surface reflectance (level2)


# specify date range: year-month-day
time_start_7='2011-08-23'
time_end_7 = '2013-02-17'

# time_start_7='2011-08-23'
# time_end_7 = '2011-09-30'

# specify date range: year-month-day
time_start_8= time_end_7
time_end_8 = '2022-01-30'
# time_start_8= '2013-02-17'
# time_end_8 = '2013-04-20'


# # sentinel-2: specify dates and cloud cover percent %
# s2=s2.filterBounds(polygon).filter('CLOUDY_PIXEL_PERCENTAGE < 100').filterDate(time_start,time_end)
# print('Total number of Sentinel-2 images:', s2.size().getInfo())
# print('band names: ', s2.first().bandNames().getInfo())
# print('info about data: ', s2.getInfo())
# print('')

# # landsat-8 (cloud cover not calculated yet) 
# l8=l8.filterBounds(polygon).filterDate(time_start,time_end)
# print('Total number of Landsat-8 images:', l8.size().getInfo())
# print('band names: ', l8.first().bandNames().getInfo())
# print('info about data: ', l8.getInfo())
# print('')

# landsat-7 (cloud cover not calculated yet) 
l7=l7.filterBounds(polygon).filter('CLOUD_COVER < 30').filterDate(time_start_7,time_end_7)
print('Total number of Landsat-7 images:', l7.size().getInfo())
print('band names: ', l7.first().bandNames().getInfo())
print('info about data: ', l7.getInfo())

# landsat-7 (cloud cover not calculated yet) 
l8=l8.filterBounds(polygon).filter('CLOUD_COVER < 30').filterDate(time_start_8,time_end_8)
print('Total number of Landsat-7 images:', l8.size().getInfo())
print('band names: ', l8.first().bandNames().getInfo())
print('info about data: ', l8.getInfo())

Total number of Landsat-7 images: 14
band names:  ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'SR_ATMOS_OPACITY', 'SR_CLOUD_QA', 'ST_B6', 'ST_ATRAN', 'ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_QA', 'ST_TRAD', 'ST_URAD', 'QA_PIXEL', 'QA_RADSAT']
info about data:  {'type': 'ImageCollection', 'bands': [], 'id': 'LANDSAT/LE07/C02/T1_L2', 'version': 1646187085097228, 'properties': {'date_range': [915148800000, 1643760000000], 'visualization_2_bands': 'SR_B7,SR_B4,SR_B2', 'visualization_2_name': 'Shortwave Infrared (742)', 'period': 0, 'type_name': 'ImageCollection', 'keywords': ['cfmask', 'cloud', 'etm', 'fmask', 'global', 'landsat', 'lasrc', 'le07', 'lst', 'reflectance', 'sr', 'usgs'], 'visualization_1_bands': 'SR_B4,SR_B3,SR_B2', 'thumb': 'https://mw1.google.com/ges/dd/images/LANDSAT_SR_thumb.png', 'visualization_2_gain': '1.8,1.9,1.9', 'description': '<p>This dataset contains atmospherically corrected\nsurface reflectance and land surface temperature derived from the dat

In [3]:
# clip images to the region defined
def clip_image (image):
  return image.clip(polygon) 

# s2=s2.map(clip_image)
l8=l8.map(clip_image)
l7=l7.map(clip_image)

# merge images from same day 
# not sure how to do this

print('Total number of Landsat-7 images:', l7.size().getInfo())
print('Total number of Landsat-7 images:', l8.size().getInfo())

Total number of Landsat-7 images: 14
Total number of Landsat-7 images: 85


In [4]:
# Import libs
import folium
from IPython.display import Image

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# collection information for loop
nimg17 = l7.size().getInfo()
collection_list = l7.toList(nimg17)
n17 = collection_list.size().getInfo()

# collection information for loop
nimg18 = l8.size().getInfo()
collection_list = l8.toList(nimg18)
n18 = collection_list.size().getInfo()

Plot the true color images.

The following code plots the landsat8 images in a loop.


In [5]:


# for i in range(0,n17): #n17
#   img = ee.Image(collection_list.get(i));
#   # print the date
#   id = img.id().getInfo();
#   print(id)
#   # Add EE drawing method to folium.
#   folium.Map.add_ee_layer = add_ee_layer
#   # Set visualization parameters.
#   vis_params = {
#               'min': 0,
#               'max': 0.3,
#               'dimensions': 512,
#               'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
#               'region': polygon}

#   # Create a folium map object.
#   my_map = folium.Map(location=[42.926350,-123.444683], zoom_start=10, height=800, width=1000)
#   # Add the image to the map object. If want to display the entire region, remove .clip(polygon)
#   my_map.add_ee_layer(img.clip(polygon), vis_params, 'bands')  # 
#   # Add a layer control panel to the map.
#   my_map.add_child(folium.LayerControl())
#   # Display the map.
#   display(my_map)


In [6]:
# set up for exporting image collections to google drive

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

# defining an function to export each image in the clipped collection
def ExportCol(col, folder, scale, tp, maxPixels, region):
  tp = 'float' 
  scale = 10 
  maxPixels =  100e9;
  nimg=col.size().getInfo()
  colList = col.toList(nimg);
  n = colList.size().getInfo();

  for i in range(0,n):
      img = ee.Image(colList.get(i)).select(['SR_B4', 'SR_B3', 'SR_B2']);
      id = img.id().getInfo();
      region = region # img.geometry().bounds().getInfo()["coordinates"];

      imgtype = {"float":img.toFloat(), 
                 "byte":img.toByte(), 
                 "int":img.toInt(),
                 "double":img.toDouble()
                }

      task=ee.batch.Export.image.toDrive(
        image=imgtype[tp],
        description= id,
        folder= folder,
        fileNamePrefix= id,
        region= region,
        scale= scale,
        maxPixels= maxPixels)
      task.start()

Mounted at /content/gdrive


In [ ]:
# Export image collections to google drive
ExportCol(l7,'GEOG_L7',10,'float',100e9,polygon)

In [ ]:
# set up for exporting image collections to google drive

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

# defining an function to export each image in the clipped collection
def ExportCol(col, folder, scale, tp, maxPixels, region):
  tp = 'float' 
  scale = 10 
  maxPixels =  100e9;
  nimg=col.size().getInfo()
  colList = col.toList(nimg);
  n = colList.size().getInfo();

  for i in range(0,n):
      img = ee.Image(colList.get(i)).select(['SR_B5', 'SR_B4', 'SR_B3']);
      id = img.id().getInfo();
      region = region # img.geometry().bounds().getInfo()["coordinates"];

      imgtype = {"float":img.toFloat(), 
                 "byte":img.toByte(), 
                 "int":img.toInt(),
                 "double":img.toDouble()
                }

      task=ee.batch.Export.image.toDrive(
        image=imgtype[tp],
        description= id,
        folder= folder,
        fileNamePrefix= id,
        region= region,
        scale= scale,
        maxPixels= maxPixels)
      task.start()

In [ ]:
# Export image collections to google drive
ExportCol(l8,'GEOG_L8',10,'float',100e9,polygon)